In [1]:
!pip install opencv-contrib-python==3.4.2.17

import cv2
import numpy as np 
import keras
from keras.datasets import cifar10
import itertools
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.metrics import accuracy_score
from sklearn.neighbors import BallTree
import pickle
import glob
from sklearn.preprocessing import StandardScaler 
from sklearn.decomposition import PCA 

     |████████████████████████████████| 30.6MB 149kB/s 
  Found existing installation: opencv-contrib-python 4.1.2.30
    Uninstalling opencv-contrib-python-4.1.2.30:
      Successfully uninstalled opencv-contrib-python-4.1.2.30


In [2]:
# We are loading cifar10 data from keras
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print(y_test[0],'y_test')

170500096/170498071 [==============================] - 4s 0us/step
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
[3] y_test


In [3]:
def desSIFT(image):
    sift = cv2.xfeatures2d.SIFT_create()
    kp, des = sift.detectAndCompute(image,None)		
    #img2 = cv2.drawKeypoints(img,kp,None,(0,255,0),4)
    #plt.imshow(img2),plt.show()
    return kp,des

def describeSURF( image):
    surf = cv2.xfeatures2d.SURF_create()
    surf.setHessianThreshold(400)
    kp, des = surf.detectAndCompute(image,None)
    return kp,des

def describeORB( image):
    orb=cv2.ORB_create()
    kp, des=orb.detectAndCompute(image,None)
    return kp,des



In [4]:
def getDescriptors(images) : 
    descriptors = []
    
    for image in images : 
        print (image.shape)
        # Re-sizing the image
        image = cv2.resize(image, (150, 150), interpolation=cv2.INTER_AREA)
        # Converting the image into gray scale
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
        kp, des = describeSURF(image)
        if des is not None : 
            descriptors.append(des)
            
    descriptors = np.concatenate(descriptors, axis=0)
    descriptors = np.asarray(descriptors)
        
    return descriptors

def getVLADDescriptors(images, images_lables, visualDic):
    descriptors = []
    labels = []
    
    count = 0
    for image in images :
        # Re-sizing the image
        image = cv2.resize(image, (150, 150), interpolation=cv2.INTER_AREA) 
        # Converting the image into gray scale
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
        kp, des = describeSURF(image)
        if des is not None : 
            v = VLAD(des, visualDic)
            descriptors.append(v)
            labels.append(images_lables[count])
        count += 1
            
            
    descriptors = np.asarray(descriptors)
    print(labels)
    return descriptors, labels

def VLAD(X, visualDictionary) : 
    
    predictedLabels = visualDictionary.predict(X)
    centers = visualDictionary.cluster_centers_
    labels = visualDictionary.labels_
    k = visualDictionary.n_clusters
    
    m,d = X.shape
    V=np.zeros([k,d])

    for i in range(k):
        if np.sum(predictedLabels==i)>0:
            V[i]=np.sum(X[predictedLabels==i,:]-centers[i],axis=0)
    

    V = V.flatten()
    V = np.sign(V)*np.sqrt(np.abs(V))

    V = V/np.sqrt(np.dot(V,V))
    return V

In [5]:
surf_des = getDescriptors(np.concatenate((x_train, x_test), axis = 0))
print (surf_des.shape)

Streaming output truncated to the last 5000 lines.
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(

In [9]:
# using subset k means to divide it into clusters size k on surf descriptors
k = 30
visDic = MiniBatchKMeans(init='k-means++', n_clusters=30,max_iter=1000, batch_size=1000, n_init=10, max_no_improvement=10, verbose=0).fit(surf_des)

In [10]:
# getting vlad descriptors on the train data
vlad_des, labels = getVLADDescriptors(x_train, y_train, visDic)
# getting vlad descriptors on the test data
vlad_des_test, labels_test = getVLADDescriptors(x_test, y_test, visDic)
#print  (vlad_des_test)
#print (labels_test)

[array([6], dtype=uint8), array([9], dtype=uint8), array([9], dtype=uint8), array([4], dtype=uint8), array([1], dtype=uint8), array([1], dtype=uint8), array([2], dtype=uint8), array([7], dtype=uint8), array([8], dtype=uint8), array([3], dtype=uint8), array([4], dtype=uint8), array([7], dtype=uint8), array([7], dtype=uint8), array([2], dtype=uint8), array([9], dtype=uint8), array([9], dtype=uint8), array([9], dtype=uint8), array([3], dtype=uint8), array([2], dtype=uint8), array([6], dtype=uint8), array([4], dtype=uint8), array([3], dtype=uint8), array([6], dtype=uint8), array([6], dtype=uint8), array([2], dtype=uint8), array([6], dtype=uint8), array([3], dtype=uint8), array([5], dtype=uint8), array([4], dtype=uint8), array([0], dtype=uint8), array([0], dtype=uint8), array([9], dtype=uint8), array([1], dtype=uint8), array([3], dtype=uint8), array([4], dtype=uint8), array([0], dtype=uint8), array([3], dtype=uint8), array([7], dtype=uint8), array([3], dtype=uint8), array([3], dtype=uint8),

In [11]:
# Performing pca n vlad descriptors
sc = StandardScaler() 
pca = PCA(n_components = 0.7) 

vlad_des = sc.fit_transform(vlad_des) 
vlad_des_test = sc.transform(vlad_des_test) 
vlad_des = pca.fit_transform(vlad_des) 
vlad_des_test = pca.transform(vlad_des_test)

In [12]:
clf = cv2.ml.KNearest_create()
clf.train(np.float32(vlad_des), cv2.ml.ROW_SAMPLE, np.float32(labels))

True

In [13]:
ret, results, neighbours ,dist = clf.findNearest(np.float32(vlad_des_test), k=10)

In [14]:
pred_label = []
correct_label = []

for label in results:
    pred_label.append(int(label))

for label in labels_test:
    correct_label.append(int(label))

#print (y_test)
#print (pred_label)

print (len(pred_label))
print (len(correct_label))
    
accuracy_score(labels_test, pred_label)

10000
10000


0.3292